In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Silver Data Transformation

In [0]:
df = spark.read.format("delta")\
         .option("header", "true")\
         .option("inferSchema", "true")\
         .load("abfss://bronze@netflixprojstorageacc.dfs.core.windows.net/netflix_titles")

In [0]:
df.display()

In [0]:
df = df.fillna({"duration_minutes": 0, "duration_seasons": 1})

In [0]:
df.display()

In [0]:
df=df.withColumn("duration_minutes",col('duration_minutes').cast(IntegerType()))\
    .withColumn("duration_seasons",col('duration_seasons').cast(IntegerType()))

In [0]:
df.printSchema()

root
 |-- duration_minutes: integer (nullable = true)
 |-- duration_seasons: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- description: string (nullable = true)
 |-- show_id: string (nullable = true)
 |-- _rescued_data: string (nullable = true)



In [0]:
df = df.withColumn("Title_Start",split(col("title"), ":")[0])
df.display()

In [0]:
df = df.withColumn("rating",split(col('rating'), "-")[0])
df.display()

In [0]:
df = df.withColumn("type_flag",when(col('type')=='Movie', 1).otherwise(0))
display(df)

In [0]:
from pyspark.sql.window import Window

In [0]:
df = df.withColumn("duration_ranking",dense_rank().over(Window.orderBy(col('duration_minutes').desc())))

/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
df.display()

In [0]:
df.createOrReplaceTempView("temp_view")

/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
df=spark.sql("SELECT * FROM temp_view")

In [0]:
df.display()

In [0]:
df_viz = df.groupBy("type").agg(count("*").alias("total_count"))
display(df_viz)

In [0]:
df.write.format("delta")\
        .mode("overwrite")\
        .option("path","abfss://silver@netflixprojstorageacc.dfs.core.windows.net/netflix_titles")\
        .save()